In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
transit_jobs_access = pd.read_csv(output_path / 'access/transit_jobs_access.csv', 
                                  usecols=['geography', 'value', 'geography_group'])
walk_bike_jobs_access = pd.read_csv(output_path / 'access/walk_bike_jobs_access.csv', 
                                  usecols=['geography_value', 'jobs_1_mile_walk', 'jobs_3_mile_bike', 'geography_group']).\
                                  rename(columns={'geography_value': 'geography'})

parcel_emp = pl.read_csv(output_path / 'landuse/parcels_urbansim.txt', 
                         separator=' ', 
                         columns=['parcelid','emptot_p']).to_pandas()


# jobs access in equity geographies
equity_geogs = summary_config['equity_geogs']

## Jobs Accessible within 45 Minutes of Transit

In [4]:
def process_access_data(jobs_access):
    df_access = jobs_access.copy()
    # rename region
    df_access.loc[jobs_access['geography_group'] == 'region', 'geography'] = 'Region'
    # rename rgc
    df_access.loc[jobs_access['geography_group'] == 'rgc_binary', 'geography'] = ['Not in RGC', 'In RGC']

    df_access_equity = df_access.loc[
        df_access['geography_group'].isin(equity_geogs)].copy()
    
    df_access_equity['geography'] = df_access_equity['geography'].map({"0.0": 'Below Regional Average', 
                                                                           "1.0": 'Above Regional Average', 
                                                                           "2.0": 'Higher Share of Equity Population'}
                                                                           )

    # df_access_equity_geogs['geography'] = df_access_equity_geogs['geography_group']
    # df['geography'] = "NOT in " + df['geography_group']

    # df_access_equity_geogs = pd.concat([df_access_equity_geogs, df], ignore_index=True)
    # df_access_equity_geogs['geography_group'] = 'Equity Geography'

    return df_access, df_access_equity


df_access_t, df_access_equity_t = process_access_data(transit_jobs_access)
df_access_bp, df_access_equity_bp = process_access_data(walk_bike_jobs_access)
tot_jobs = parcel_emp['emptot_p'].sum()

In [5]:
def job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin([geog, 'region'])].\
        rename(columns={'value': 'Jobs within 45-minute Transit Commute'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs'] = df['Jobs within 45-minute Transit Commute'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df


In [6]:
df = job_access_geog(df_access_t,'CountyName')
df = df[df.index != 'Outside Region']
df

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
King,"231,151",10.7%
Kitsap,"7,049",0.3%
Pierce,"19,366",0.9%
Snohomish,"26,491",1.2%
Region,"136,703",6.3%


In [7]:
df_rgc = job_access_geog(df_access_t,'rgc_binary')
df = job_access_geog(df_access_t,'GrowthCenterName')

pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"136,703",6.3%
Not in RGC,"101,137",4.7%
In RGC,"471,659",21.9%
Auburn,"66,565",3.1%
Bellevue,"472,521",21.9%
Bothell Canyon Park,"38,963",1.8%
Bremerton,"39,381",1.8%
Burien,"121,961",5.7%
Everett,"61,987",2.9%


In [8]:
job_access_geog(df_access_t,'rg_proposed')

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Region,"136,703",6.3%
CitiesTowns,"10,463",0.5%
Core,"58,381",2.7%
HCT,"36,728",1.7%
Metro,"337,627",15.7%
UU,"10,058",0.5%


In [9]:
df = pd.DataFrame()
for label, col in {
    "People of Color": "equity_focus_areas_2023__efa_poc",
    "Income": "equity_focus_areas_2023__efa_pov200",
    "LEP": "equity_focus_areas_2023__efa_lep",
    "Disability": "equity_focus_areas_2023__efa_dis",
    "Older Adults": "equity_focus_areas_2023__efa_older",
    "Youth": "equity_focus_areas_2023__efa_youth"
    }.items():
    _df = job_access_geog(df_access_equity_t, col)
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'geography': 'EFA Type'}, inplace=True)
df[['Group', 'EFA Type', 'Jobs within 45-minute Transit Commute', '% Total Jobs']]

,Group,EFA Type,Jobs within 45-minute Transit Commute,% Total Jobs
0,People of Color,Below Regional Average,"120,125",5.6%
1,People of Color,Above Regional Average,"154,036",7.1%
2,People of Color,Higher Share of Equity Population,"160,399",7.4%
3,Income,Below Regional Average,"137,951",6.4%
4,Income,Above Regional Average,"128,464",6.0%
5,Income,Higher Share of Equity Population,"146,239",6.8%
6,LEP,Below Regional Average,"144,583",6.7%
7,LEP,Above Regional Average,"125,792",5.8%
8,LEP,Higher Share of Equity Population,"120,258",5.6%
9,Disability,Below Regional Average,"158,226",7.3%


## Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.

Average accessible jobs are weighted averages based on parcel household population.

In [10]:
def bp_job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin(['region', geog])].\
        rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs (1-mile walk)'] = df['Jobs within 1-mile Walk'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')
    df['% Total Jobs (3-mile bike)'] = df['Jobs within 3-mile Bike'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df

In [11]:
df = bp_job_access_geog(df_access_bp,'CountyName')
df = df[df.index != 'Outside Region']
df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
King,"18,516","84,913",0.9%,3.9%
Kitsap,"1,236","8,014",0.1%,0.4%
Pierce,"2,508","18,163",0.1%,0.8%
Snohomish,"2,006","17,703",0.1%,0.8%
Region,"11,164","54,254",0.5%,2.5%


In [12]:
df_rgc = bp_job_access_geog(df_access_bp,'rgc_binary')
df = bp_job_access_geog(df_access_bp,'GrowthCenterName')

pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"11,164","54,254",0.5%,2.5%
Not in RGC,"2,912","34,868",0.1%,1.6%
In RGC,"88,878","236,824",4.1%,11.0%
Auburn,"10,541","40,324",0.5%,1.9%
Bellevue,"59,188","110,699",2.7%,5.1%
Bothell Canyon Park,"8,539","21,748",0.4%,1.0%
Bremerton,"11,536","34,387",0.5%,1.6%
Burien,"4,829","13,400",0.2%,0.6%
Everett,"15,908","39,722",0.7%,1.8%


In [13]:
bp_job_access_geog(df_access_bp,'rg_proposed')

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"11,164","54,254",0.5%,2.5%
CitiesTowns,984,"8,948",0.0%,0.4%
Core,"3,415","28,587",0.2%,1.3%
HCT,"1,428","15,237",0.1%,0.7%
Metro,"29,432","128,074",1.4%,5.9%
UU,443,"6,816",0.0%,0.3%


In [14]:
df = pd.DataFrame()
for label, col in {
    "People of Color": "equity_focus_areas_2023__efa_poc",
    "Income": "equity_focus_areas_2023__efa_pov200",
    "LEP": "equity_focus_areas_2023__efa_lep",
    "Disability": "equity_focus_areas_2023__efa_dis",
    "Older Adults": "equity_focus_areas_2023__efa_older",
    "Youth": "equity_focus_areas_2023__efa_youth"
    }.items():
    _df = bp_job_access_geog(df_access_equity_bp, col)
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'geography': 'EFA Type'}, inplace=True)
df[['Group', 'EFA Type', 'Jobs within 1-mile Walk',	'Jobs within 3-mile Bike',	'% Total Jobs (1-mile walk)','% Total Jobs (3-mile bike)']]

,Group,EFA Type,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
0,People of Color,Below Regional Average,"5,374","41,935",0.2%,1.9%
1,People of Color,Above Regional Average,"18,509","71,731",0.9%,3.3%
2,People of Color,Higher Share of Equity Population,"17,229","63,994",0.8%,3.0%
3,Income,Below Regional Average,"9,394","52,420",0.4%,2.4%
4,Income,Above Regional Average,"13,899","56,258",0.6%,2.6%
5,Income,Higher Share of Equity Population,"13,651","58,320",0.6%,2.7%
6,LEP,Below Regional Average,"11,368","57,818",0.5%,2.7%
7,LEP,Above Regional Average,"13,284","51,842",0.6%,2.4%
8,LEP,Higher Share of Equity Population,"7,745","43,674",0.4%,2.0%
9,Disability,Below Regional Average,"9,651","56,870",0.4%,2.6%


## Intersection Density

In [15]:
buffered_parcels = pl.read_csv(output_path / 'landuse/buffered_parcels.txt', 
                               separator=' ',
                               columns=['parcelid','nodes3_2','nodes4_2','hh_p'])


async_engine = create_engine('sqlite:///' + summary_config['sc_run_path'] + '/inputs/db/' + config['db_name'])

list_cols = ['ParcelID','CountyName','GrowthCenterName','rg_proposed'] + equity_geogs
parcel_geog = pl.read_database(
    query= f"SELECT {', '.join(list_cols)} FROM " + "parcel_" + config["base_year"] + "_geography",
    connection=async_engine.connect()
)

In [16]:
df_intersection = buffered_parcels.join(parcel_geog, left_on='parcelid', right_on='ParcelID').to_pandas()

# Total intersections within 1/2 mile buffer
df_intersection['intersections_wt'] = (df_intersection['nodes3_2'] + df_intersection['nodes4_2']) * df_intersection['hh_p']

In [17]:
def intersection_density(geog, map=False):
    df = df_intersection.groupby(geog)[['intersections_wt', 'hh_p']].sum().reset_index()
    df['Intersections'] = df['intersections_wt']/df['hh_p']

    if map:
        df[geog] = df[geog].astype('int').map({0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}
                                )
    
    return df[[geog] + ['Intersections']]

In [18]:
df = intersection_density('CountyName')
df = df[df['CountyName']!='Outside Region']
df

,CountyName,Intersections
0,King,155
1,Kitsap,51
3,Pierce,80
4,Snohomish,73


In [19]:
intersection_density('GrowthCenterName')

,GrowthCenterName,Intersections
0,Auburn,192
1,Bellevue,273
2,Bothell Canyon Park,69
3,Bremerton,168
4,Burien,176
5,Everett,161
6,Federal Way,134
7,Greater Downtown Kirkland,167
8,Issaquah,NaN
9,Kent,214


In [20]:
intersection_density('rg_proposed')

,rg_proposed,Intersections
0,Cities and Towns,65
1,Core Cities,103
2,High Capacity Transit Communities,84
3,Metropolitan Cities,202
4,Rural Areas,20
5,Urban Unincorporated Areas,56


In [21]:
intersection_density('equity_focus_areas_2023__efa_poc')

,equity_focus_areas_2023__efa_poc,Intersections
0,0,103
1,1,136
2,2,136


In [22]:
df = pd.DataFrame()
for label, col in {
    "People of Color": "equity_focus_areas_2023__efa_poc",
    "Income": "equity_focus_areas_2023__efa_pov200",
    "LEP": "equity_focus_areas_2023__efa_lep",
    "Disability": "equity_focus_areas_2023__efa_dis",
    "Older Adults": "equity_focus_areas_2023__efa_older",
    "Youth": "equity_focus_areas_2023__efa_youth"
    }.items():
    _df = intersection_density(col).rename(columns={col: "Group"})
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })
df[['Group', 'EFA Type', 'Intersections']]

,Group,EFA Type,Intersections
0,People of Color,Below Regional Average,103
1,People of Color,Above Regional Average,136
2,People of Color,Higher Share of Equity Population,136
3,Income,Below Regional Average,112
4,Income,Above Regional Average,122
5,Income,Higher Share of Equity Population,137
6,LEP,Below Regional Average,117
7,LEP,Above Regional Average,120
8,LEP,Higher Share of Equity Population,121
9,Disability,Below Regional Average,121
